# Overload signatures

Functions can have multiple combination of arguments

Use `@typing.overload` in order to allow type annotation of these combinations. This is really helpful when the returning type depends on one or more input parameters. For example the builtin function sum has the following signature:

```python
@overload
def sum(__iterable: Iterable[_T]) -> Union[_T, int]: ... # Why int?
@overload
def sum(__iterable: Iterable[_T], start: _S) -> Union[_T, _S]: ...
```

According to the function docs, if the iterable is empty, the value returned is the `start` value, hence, the return type should be the union of `_T` and `_S`. 



In [1]:
# sum implementation to show how to use overload

import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...
def sum(it, /, start=0):
    return functools.reduce(operator.add, it, start)


In [2]:
from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
DT = TypeVar('DT')
LT = TypeVar('LT', bound=SupportsLessThan)

MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'

@overload # key: None = ... ?
def max(__arg1: LT, __arg2: LT, *args:LT, key: None = ...) -> LT: ...
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T: ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ..., default: DT) -> Union[LT, DT]: ...
def max(first, *args, key=None, default=MISSING):
    if args:
        series = args
        candidate = iter(first)
    else:
        series = iter(first)
        try:
            candidate = next(series)
        except StopIteration:
            if default is not MISSING:
                return default
            raise ValueError(EMPTY_MSG) from None
    if key is None:
        for current in series:
            if candidate < current:
                candidate = current
    else:
        candidate_key = key(candidate)
        for current in series:
            current_key = key(current)
            if candidate_key < current_key:
                candidate = current
                candidate_key = current_key
    return candidate

# TypedDict

When trying to add type hints to a dict, we face a problem if the value types are heterogeneous. For example, trying to type hint the following dictionary

```python
{
    "isbn": "0134757599",
    "title": "Refactoring, 2e",
    "authors": ["Martin Fowler", "Kent Beck"],
    "pagecount": 478
}
```

would require something like `Dict[str, Any]` or `Dict[str, Union[str, int, List[str]]]`. Both cases are not good because the first does not tell a lot about the types and the second does not state for which field the type is about.

Here we could use a `TypedDict`:

```python
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str,
    title: str,
    authors: list[str]
    pagecount: int

pp = BookDict(title='Programming Pearls',
            authors='Jon bentley', # Not a list, but the interpreter would not throw a error because of gradual typing
            isbn='00000000',
            pagecount=256)

pp['title'] # 'Programming Pearls
pp.title # AttributeError

```

Notice that the syntax is close to a `NamedTuple` or a `dataclass` but the TypedDict is a little bit worse, because it does not support runtime verifications and some other utilities.

So, in most cases, specially when dealing with code that deals with data input from a json for example, we need to be careful when using a `TypedDict`.

# Type Coercion (Forcing a type to be another type)

The type system is not perfect, sometimes we need to use brute force. Careful!

The special function `typing.cast()` is used to convert a type of something

```python
from typing import cast

def find_first_str(a: list[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))

    # return a[index] - mypy would have a problem with this because 'a[index]' type is object
    return cast(str, a[index]) 
```

The implementation of `typing.cast` does nothing to the input parameter. The only thing that makes the coercion is that mypy will believe blindly in the first argument of the `cast()` function.

If you use a lot the `typing.cast` function, it could indicate a code smell, because a lot of times mypy is correct about your typing problem.

Alternatives to use of `typing.cast` is `# type : ignore` comment in the end of a line or using `Any` for everything becuase `Any` is compatible with all types.

# Reading type hints during execution

During importation of a module, Python will store the type hints in attributes called `__annotations__`.

For example, consider the function below:

```python
def clip(text: str, max_len: int = 80) -> str: ...

clip.__annotation__ # {'text': str, 'max_len': int, 'return': str}
```

There are two problems with this:
- storage and cpu use to deal with this info
- references to types yet to be define will be done with strings

The second problem happens when we are importing a module, then a function uses a object that have not being imported yet. This will cause this object to be represented as a string in the `__annotations__` dict. This happens when we are defining a member function that returns the type of the class. For example

```python
class Rectangle:
    width: float

    def stretch(self, factor: float) -> 'Rectangle':
        return Rectangle(width=self.width * factor)
```

However, if we try to get the `__annotations__` for `strech` we would receive as return type `'Rectangle'`  instead of the class Rectangle.

To get this info properly, use `typing.get_type_hints()` for Python 3.5 until 3.9 or `typing.get_annotations()` for Python 3.10 forward.

This is still being debated by the python community, so be careful when using this. It is recommended to create a class that deals with this and if the behavior of the language changes, we need to update this only in one place. For example,

```python
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)
```



# Implementing a generic class

Let's update the class `LottoBlower` to be generic, i. e., be able to use a specific type for it's iterator argument. This type is informed by the user of the class.

## Jargon to the generic types

- Generic type: A type declared with one or more type variable. Example: `LottoBlower[T]`, `abc.Mapping[KT, VT]`
- Formal type parameter: the type variables that shows up in the declaration of a generic type. Example: `KT`, `VT`
- Parametrized type: A generic type filled with real types. Example: `LottoBlower[int]`
- Real type parameter: the real type passed to the parametrized type. Example: `int` in the `LottoBlower[int]`

In [6]:
import random

from collections.abc import Iterable
from typing import TypeVar, Generic

from tombola import Tombola # The interface defined on chapter 13

T = TypeVar('T')

class LottoBlower(Tombola, Generic[T]):

    def __init__(self, items: Iterable[T]) -> None:
        self._balls = list[T](items)
    
    def load(self, items: Iterable[T]) -> None:
        self._balls.extend(items)

    def pick(self) -> T:
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)
    
    def loaded(self) -> bool:
        return bool(self._balls)

    def inspect(self) -> tuple[T, ...]:
        return tuple(self._balls)
    
good_machine = LottoBlower[int]([1, 2, 3])
bad_machine1 = LottoBlower[int]([1, 0.2]) # incompatible format float
good_machine.load('ABC') # incompatible type str, expected int




# Variance

Too complex to describe, I've just read the book.

By default `TypeVar` will create a type invariant, we can make them covariant or contravariant by using a flag to the constructor.